In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import os
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import warnings
warnings.filterwarnings('ignore')

In [19]:
df1 = pd.read_csv('Train_NY_cleaned.csv')
df1.head(1)

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,0,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,392,6,10009,5,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19


In [20]:
df2 = pd.read_csv('Test_NY_cleaned.csv')
df2.head(1)

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,399,26,10009,28,31,4616.0,18690.0,1900,2,C7,NaN,2016-12-14


In [21]:
df1['Data']='Train'
df2['Data']='Test'

In [22]:
dff = pd.concat([df1,df2])
dff.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,Data
0,0,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,392,6,10009,5,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19,Train
1,3,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,402,21,10009,10,10,2272.0,6794.0,1913,2,C4,3936272.0,2016-09-23,Train
2,4,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,404,55,10009,6,6,2369.0,4615.0,1900,2,C2,8000000.0,2016-11-17,Train
3,6,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,406,32,10009,8,8,1750.0,4226.0,1920,2,C4,3192840.0,2016-09-23,Train
4,9,1,ALPHABET CITY,RENTALS - ELEVATOR APARTMENTS,387,153,10009,24,24,4489.0,18523.0,1920,2,D9,16232000.0,2016-11-07,Train


In [23]:
dff.shape

(84548, 17)

In [24]:
dff=dff.drop(['Unnamed: 0','NEIGHBORHOOD','BUILDING CLASS CATEGORY'],axis=1)

In [25]:
dff['BOROUGH']=dff['BOROUGH'].replace({1:'Manhattan',2:'Bronx',3:'Brooklyn',4:'Queens',5:'Staten Island'})

In [26]:
dff.head(1)

,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,Data
0,Manhattan,392,6,10009,5,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19,Train


In [27]:
dff['SALE YEAR']=dff['SALE DATE'].apply(lambda x:x[:4])
dff['SALE YEAR']= dff['SALE YEAR'].astype(int)

In [28]:
dff=dff.drop('SALE DATE',axis=1)

In [29]:
dff['Property Age'] = dff['SALE YEAR'] - dff['YEAR BUILT']
dff['Property Age'] = dff['SALE YEAR'] - dff['YEAR BUILT']

In [30]:
dff=dff.drop('YEAR BUILT',axis=1)

In [31]:
dff['BUILDING CLASS AT TIME OF SALE'].value_counts().head(10)

R4    12989
D4    12666
A1     6751
A5     5671
B2     4918
B1     4747
C0     4384
B3     3821
A2     2867
C6     2760
Name: BUILDING CLASS AT TIME OF SALE, dtype: int64

In [32]:
cat_cols = dff.select_dtypes('object')
cat_cols=cat_cols.columns[:-1]
cat_cols

Index(['BOROUGH', 'BUILDING CLASS AT TIME OF SALE'], dtype='object')

In [33]:
for col in cat_cols:
    freqs=dff[col].value_counts()
    k=freqs.index[freqs>2500][:-1]
    for cat in k:
        name=col+'_'+cat
        dff[name]=(dff[col]==cat).astype(int)
    del dff[col]
    print(col)

BOROUGH
BUILDING CLASS AT TIME OF SALE


In [34]:
dff.columns

Index(['BLOCK', 'LOT', 'ZIP CODE', 'RESIDENTIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'TAX CLASS AT TIME OF SALE',
       'SALE PRICE', 'Data', 'SALE YEAR', 'Property Age', 'BOROUGH_Queens',
       'BOROUGH_Brooklyn', 'BOROUGH_Manhattan', 'BOROUGH_Staten Island',
       'BUILDING CLASS AT TIME OF SALE_R4',
       'BUILDING CLASS AT TIME OF SALE_D4',
       'BUILDING CLASS AT TIME OF SALE_A1',
       'BUILDING CLASS AT TIME OF SALE_A5',
       'BUILDING CLASS AT TIME OF SALE_B2',
       'BUILDING CLASS AT TIME OF SALE_B1',
       'BUILDING CLASS AT TIME OF SALE_C0',
       'BUILDING CLASS AT TIME OF SALE_B3',
       'BUILDING CLASS AT TIME OF SALE_A2'],
      dtype='object')

In [35]:
dff_train=dff[dff['Data']=='Train']
del dff_train['Data']
dff_test=dff[dff['Data']=='Test']
dff_test.drop(['SALE PRICE','Data'],axis=1,inplace=True)

In [36]:
dff_train.shape

(59759, 24)

In [37]:
dff_test.shape

(24789, 23)

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
dff_train1,dff_train2 = train_test_split(dff_train,test_size=0.2,random_state=42)

print(dff_train1.shape)
print(dff_train2.shape)

(47807, 24)
(11952, 24)


In [40]:
x_train1 = dff_train1.drop('SALE PRICE',axis=1)
y_train1 = dff_train1['SALE PRICE']

In [41]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train1,y_train1)

LinearRegression()

In [42]:
from sklearn.metrics import r2_score

In [43]:
x_test1 = dff_train2.drop('SALE PRICE',axis=1)
predicted = lr.predict(x_test1)

In [44]:
y_test1=dff_train2['SALE PRICE']

In [45]:
# base model r2_score for train and test


print('Train R2 Score:',lr.score(x_train1,y_train1))
print('Test R2 Score:',lr.score(x_test1,y_test1))

Train R2 Score: 0.29600041010483047
Test R2 Score: 0.34415711278329875


In [46]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

dtr = DecisionTreeRegressor()

dtr.fit(x_train1,y_train1)


DecisionTreeRegressor()

In [30]:
dtr.score(x_train1,y_train1)

0.998226469378868

In [31]:
dtr.score(x_test1,y_test1)

0.6753772093388473

In [ ]:
# RFE

In [73]:
# Checking for n_features number

no=23
score_train=[]
score_test=[]

for i in range(1,no+1):
    dtr=DecisionTreeRegressor()
    rfe=RFE(dtr,n_features_to_select=i)
    rfe.fit(x_train1,y_train1)
    
    score1=rfe.score(x_train1,y_train1)
    score_train.append(score1)
    
    score2=rfe.score(x_test1,y_test1)
    score_test.append(score2)
print(score_train)
print(score_test)

[0.8950112908115405, 0.9260164991438328, 0.9674554005418429, 0.9736991316816446, 0.9743891555364727, 0.9982263242996251, 0.9982263242996251, 0.9982264581127946, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868, 0.998226469378868]
[0.6558592682997392, -0.01196853753546434, 0.042109488829847996, 0.05582861767303127, -0.01139600908473093, -0.017808855724142925, -0.02767767250206643, 0.023894723207942947, 0.04343716999459879, -0.01653919030910389, -0.01420109954182669, -0.0072511342665591005, -0.013386443775887491, 0.6753022559948354, 0.46946602684308736, 0.04625782619371832, 0.6725371463951513, 0.6776521913793296, 0.04715305160490402, 0.046964193282149846, -0.03257622189022458, 0.3924870464010489, -0.008087898004244032]


In [89]:
rfe=RFE(dtr)

params={'n_features_to_select': list(range(1,24))}

rsearch=RandomizedSearchCV(rfe,param_distributions=params, scoring='r2', cv=3)
rsearch.fit(x_train1,y_train1)

RandomizedSearchCV(cv=3, estimator=RFE(estimator=DecisionTreeRegressor()),
                   param_distributions={'n_features_to_select': [1, 2, 3, 4, 5,
                                                                 6, 7, 8, 9, 10,
                                                                 11, 12, 13, 14,
                                                                 15, 16, 17, 18,
                                                                 19, 20, 21, 22,
                                                                 23]},
                   scoring='r2')

In [90]:
pd.DataFrame(rsearch.cv_results_).iloc[:,5:].head(3)

,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,{'n_features_to_select': 11},-0.346610,0.053393,0.612132,0.106305,0.393189,3
1,{'n_features_to_select': 22},-0.427528,0.077009,0.415979,0.021820,0.346564,7
2,{'n_features_to_select': 6},-0.378130,0.151208,0.516556,0.096545,0.367294,5


In [92]:
n_features_optimal=11

dtr=DecisionTreeRegressor()
rfe=RFE(dtr,n_features_to_select=11)
rfe.fit(x_train1,y_train1)

RFE(estimator=DecisionTreeRegressor(), n_features_to_select=11)

In [94]:
r=df(rfe.ranking_,index=x_train1.columns,columns=['Rank']).sort_values(by='Rank')
r

,Rank
BLOCK,1
LOT,1
ZIP CODE,1
RESIDENTIAL UNITS,1
TOTAL UNITS,1
LAND SQUARE FEET,1
GROSS SQUARE FEET,1
TAX CLASS AT TIME OF SALE,1
SALE YEAR,1
Property Age,1


In [101]:
rfe_f=list(r.head(11).index)
xtrain_rfe=x_train1[rfe_f]
xtest_rfe=x_test1[rfe_f]

In [ ]:
# tuning

In [47]:
from sklearn.model_selection import RandomizedSearchCV

In [102]:
dtr = DecisionTreeRegressor()
 
params = {'max_depth':np.arange(1,30,3),
        'min_samples_split':np.arange(20,70,2),
       'min_samples_leaf':np.arange(1,15,1)}

rsearch = RandomizedSearchCV(dtr,param_distributions=params,n_iter=100,scoring='r2',n_jobs=-1,cv=3)

rsearch.fit(xtrain_rfe,y_train1)

RandomizedSearchCV(cv=3, estimator=DecisionTreeRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': array([ 1,  4,  7, 10, 13, 16, 19, 22, 25, 28]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_samples_split': array([20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52,
       54, 56, 58, 60, 62, 64, 66, 68])},
                   scoring='r2')

In [103]:
rsearch.best_params_

{'min_samples_split': 28, 'min_samples_leaf': 8, 'max_depth': 13}

In [106]:
dtr = DecisionTreeRegressor(**rsearch.best_params_)
dtr.fit(xtrain_rfe,y_train1)

DecisionTreeRegressor(max_depth=13, min_samples_leaf=8, min_samples_split=28)

In [109]:
dtr.score(xtrain_rfe,y_train1)

0.5120151442891626

In [110]:
dtr.score(xtest_rfe,y_test1)

0.28204945619869537

In [115]:
from sklearn.metrics import mean_squared_error as mse

In [121]:
y_train_pred = dtr.predict(xtrain_rfe)
y_test_pred = dtr.predict(xtest_rfe)
print('Train RMSE:',np.sqrt(mse(y_train1,y_train_pred)))
print('Test RMSE:',np.sqrt(mse(y_test1,y_test_pred)))

Train RMSE: 6354062.346704131
Test RMSE: 17552429.754624855
